# Ptychographic Wavefront Camera Simulation

Created in July, 2024

Created by Ni Chen (https://ni-chen.github.io/)

In [ ]:
import os
import sys
import torch
from PIL import Image

print('Work on local machine')
work_folder = os.path.dirname(os.path.abspath("__file__"))

print("torch.cuda.is_available() = ", torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

os.chdir(work_folder)

sys.path.append(f'../')

%load_ext autoreload
%autoreload 2

## Example of imaging with 3x3 layout PWC

In [ ]:
%reload_ext autoreload
from FP import *
from util import *

N_hr = 512
data_dir = '../data/'

# Load the amplitude image
image = Image.open(f'{data_dir}leaf.jpg').convert('L')
image = transforms.Resize((N_hr, N_hr))(image)
image = transforms.ToTensor()(image).squeeze()
im = image / torch.max(image)

# Generate a ground truth with random phase
x_gt = torch.exp(im) * torch.exp(1j*(2*torch.pi*(2*(torch.rand([N_hr, N_hr])-0.5))))

# Construct a PWC
fp = FourierPtychography(N_hr=N_hr, N_lr=400, aperture_num=3, aperture_shape='circ', is_band_limit=True, device=device)
fp.sampling(overlapping_ratio=0.5)
fp.show_sampling()

# Generate the measurement
fp.fp_forward(x_gt)

# Reconstruction
x_est = fp.fp_gs_gd(max_it_gs=2000, max_it_gd=1000)

In [ ]:
# Display the results
imshow_list([fp.im_meas[4].cpu().numpy(), fp.im_meas[0].cpu().numpy()])
imshow_list([fp.x.abs().cpu(), fp.x_band.abs().cpu(), x_est.abs()])
imshow_list([fp.x.angle().cpu(), (fp.x_band/fp.x_band[0,0]).angle().cpu(), (x_est/x_est[0,0]).angle()])


plt.plot(fp.ang_mse_hist, label='Angle')
plt.ylabel('MSE per element')
plt.xlabel('Iteration')
plt.legend()
plt.ylim(1, 4)
plt.show()